In [62]:
from openai import OpenAI
import json
from dotenv import find_dotenv , load_dotenv
#import requests #to fetch
import os
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
_ : bool = load_dotenv(find_dotenv())

client : OpenAI = OpenAI() 

##### We Will Use MapBox for map functionalities Using API Key from this Website

https://account.mapbox.com/ 

In [63]:


# Create a function to initialize the map and set the initial view
def initialize_map(longitude, latitude, zoom):
    mapbox_access_token = os.getenv("MAP_BOX_API")
    map = px.scatter_mapbox(
        data_frame=pd.DataFrame(),
        mapbox_style="open-street-map",
        center={"lon": longitude, "lat": latitude},
        zoom=zoom,
    )
    map.update_layout(mapbox_accesstoken=mapbox_access_token)
    return map

# Create a function to update the map center and zoom level
def update_map(map, longitude, latitude, zoom):
    map.update_layout(
        mapbox_center={"lon": longitude, "lat": latitude},
        mapbox_zoom=zoom,
    )

# Create a function to add markers to the map
def add_markers(map, longitudes, latitudes, labels):
    map.add_trace(
        go.Scattermapbox(
            lon=longitudes,
            lat=latitudes,
            mode="markers",
            text=labels,
            marker=dict(size=10)
        )
    )

### Step 1: Upload the file and Create an Assistant

In [65]:
from openai.types.beta import assistant

#Upload file with assistance purpose
file = client.files.create(
    file = open("sample_airbnb_receipt.pdf" , "rb") , 
    purpose="assistants"
)
print(file)

FileObject(id='file-AHisncdiJRzFuykG9Acyz9WK', bytes=1217, created_at=1710631112, filename='sample_airbnb_receipt.pdf', object='file', purpose='assistants', status='processed', status_details=None)


In [66]:
assistant = client.beta.assistants.create(
    name="Travel Agent",
    instructions="You are a helpful travel assistant that can write and execute code, and has access to a digital map to display information",
    model="gpt-3.5-turbo-1106",
    tools=[
        {"type": "retrieval"},
        {"type": "code_interpreter"},
        {
            "type": "function",
            "function": {
                "name": "update_map",
                "description": "Update map to center on a particular location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "longitude": {
                            "type": "number",
                            "description": "Longitude of the location to center the map on"
                        },
                        "latitude": {
                            "type": "number",
                            "description": "Latitude of the location to center the map on"
                        },
                        "zoom": {
                            "type": "integer",
                            "description": "Zoom level of the map"
                        }
                    },
                    "required": ["longitude", "latitude", "zoom"]
                }
            }
        }
    ]
)

#### Create Thread

In [67]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_o2ZTSZRunRYOVaOzxy5m92Xd', created_at=1710631113, metadata={}, object='thread')


#### Add Message to Thread

In [68]:
from openai.types.beta.threads.thread_message import ThreadMessage
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Move to the france letitude is 2344 and longitude is 332"
)

dict(message)

{'id': 'msg_94V4tjZ28ZQoV9U3VlbIENso',
 'assistant_id': None,
 'content': [MessageContentText(text=Text(annotations=[], value='Move to the france letitude is 2344 and longitude is 332'), type='text')],
 'created_at': 1710631114,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_o2ZTSZRunRYOVaOzxy5m92Xd'}

##### Create Run


In [69]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
    
)

dict(run)

{'id': 'run_RK0rADWXOJ8PAQMuABT5vBoQ',
 'assistant_id': 'asst_yPoRWMQ8ZeQezJttyegq40ly',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1710631114,
 'expires_at': 1710631714,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a helpful travel assistant that can write and execute code, and has access to a digital map to display information',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_o2ZTSZRunRYOVaOzxy5m92Xd',
 'tools': [ToolAssistantToolsRetrieval(type='retrieval'),
  ToolAssistantToolsCode(type='code_interpreter'),
  ToolAssistantToolsFunction(function=FunctionDefinition(name='update_map', description='Update map to center on a particular location', parameters={'type': 'object', 'properties': {'longitude': {'type': 'number', 'description': 'Longitude of the location to center the map on'}, 'latitude': {'type': 'number',

#### Available Functions

In [70]:
available_functions = {
    "update_map": update_map,
    "add_markers": add_markers,
    
} 

##### Polling for Updates and Calling Functions


In [75]:
import time

  # Loop until the run completes or requires action
while True:
    runStatus = client.beta.threads.runs.retrieve(thread_id=thread.id,
                                                  run_id=run.id)
    # Add run steps retrieval here for debuging
    run_steps = client.beta.threads.runs.steps.list(thread_id=thread.id, run_id=run.id)
    # show_json("Run Steps:", run_steps)
    print(runStatus.status ,'.....')

    # This means run is making a function call   
    if runStatus.status == "requires_action":
        print(runStatus.status ,'.....')
        print("Status: ", "requires_action")
        show_json("submit_tool_outputs", runStatus.required_action)
        if runStatus.required_action.submit_tool_outputs and runStatus.required_action.submit_tool_outputs.tool_calls:
            print("toolCalls present:")
            toolCalls = runStatus.required_action.submit_tool_outputs.tool_calls

            tool_outputs = []
            for toolcall in toolCalls:
                function_name = toolcall.function.name
                function_args = json.loads(toolcall.function.arguments)
                
                if function_name in available_functions:
                    
                    
                    
                        response = function_to_call( **function_args)
        
                        
                        
                        tool_outputs.append({
                                  "tool_call_id": toolcall.id,
                                  "output": response
                              })
                    
                    
            print(tool_outputs,">>>>>") 
            # Submit tool outputs and update the run
            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs)
      
    elif runStatus.status == "completed":
        # List the messages to get the response
        print("completed...........logic")
        messages: list[ThreadMessage] = client.beta.threads.messages.list(thread_id=thread.id)
        for message in messages.data:
            role_label = "User" if message.role == "user" else "Assistant"
            message_content = message.content[0].text.value
            print(f"{role_label}: {message_content}\n")
        break  # Exit the loop after processing the completed run

    elif run.status == "failed":
      print("Run failed.")
      break

    elif run.status in ["in_progress", "queued"]:
      print(f"Run is {run.status}. Waiting...")
      time.sleep(5)  # Wait for 5 seconds before checking again

    else:
      print(f"Unexpected status: {run.status}")
      break




requires_action .....
requires_action .....
Status:  requires_action
submit_tool_outputs
{"submit_tool_outputs":{"tool_calls":[{"id":"call_7zF7vVtDnPbCmnWBhtJls2PI","function":{"arguments":"{\"longitude\":332,\"latitude\":2344,\"zoom\":6}","name":"update_map"},"type":"function"}]},"type":"submit_tool_outputs"}
toolCalls present:


TypeError: update_map() missing 1 required positional argument: 'map'